# Hotdog or not - Image recognition Model
The main goal of projet is to create deep program for image recognition. In this case object objects to be recognized will be hot dogs.

In [7]:
#check if compatible GPU is available 
from torch.cuda import is_available, get_device_name

if is_available():
    print(f"The environment has a compatible GPU ({get_device_name()}) available.")
else:
    print("The environment does NOT have a compatible GPU model available.")

The environment has a compatible GPU (NVIDIA GeForce GTX 950M) available.


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas

def imshow(inp: torch.Tensor) -> None:
    '''Imshow for torch.Tensor'''
    inp = inp.cpu().numpy()
    inp = inp.transpose((1,2,0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std*inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.show()

In [16]:
#import hot dog data
import os
from typing import Tuple
from zipfile import ZipFile
from PIL import Image

from numpy import array
from pandas import read_csv
from requests import get
from torch.utils.data import Dataset




class HotDogDataset(Dataset):
    """"
    Child class of torch.utlis.data.Dataset.
    This is wrapper for mapping from hotdog/not images to the target.
    """

    def __init__(self, dir_name, transform=None) -> None:
        """
        Initialize a HotdogDataset class
        :param dir_name: The name of folder holding the data.
        :param transform:
        """
        self.transform = transform
        #Download, save and extract data if needed.
        if not os.path.isdir(os.path.join(os.getcwd(), f"{dir_name}")):
            r = get(f"https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/hotdog-not-hotdog/data/{dir_name}.zip")
            f = open(os.path.join(os.getcwd(), f"{dir_name}.zip"), mode="wb+")
            f.write(r.content)
            f.close()
            with ZipFile(os.path.join(os.getcwd(), f"{dir_name}.zip"), 'r') as zip_ref:
                zip_ref.extractall("./")
                zip_ref.close()
        # Load metadata.
        self.data = read_csv(os.path.join(os.getcwd(), dir_name, f"{dir_name}_labels.csv"))
        #Number of classes.
        self.n_classes = len(self.data['y'].unique())

    def __len__(self) -> int:
        """
        :return: The length of the training/testing dataset.
        """
        return len(self.data)
    
    def __getitem__(self, idx) -> Tuple[array, str]:
        """
        Return the input and target at a specific index of the dataset.

        :param idx: The index of the data to be returned.
        :return: Key-value pair at the specified index.
        """
        # Open corresponding Image
        image = Image.open(os.path.join(os.getcwd(), self.data.loc[idx, 'file_name']))
        # Retrieve the label
        y = self.data.loc[idx, 'y']
        # Transform the image if necessary.
        if self.transform is not None:
            image_ = self.transform(image)
            image.close()
        else:
            image_ = array(image)
            image.close()
        return image_, y
